In [22]:
# %load D:/BUDS_MID/__init__.py
"""
Created on Wed Feb 19 16:21:00 2020

@author: cmchico

usual imports
"""

import os
import pandas as pd
import numpy as np

pd.options.display.max_rows = 100
pd.set_option('display.float_format', '{:.2f}'.format)

from time import time
from datetime import timedelta
import datetime

import pickle

import f_visuals as fviz
import f_prophet as fpro

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

'\nCreated on Wed Feb 19 16:21:00 2020\n\n@author: cmchico\n\nusual imports\n'

In [23]:
# %load D:/BUDS_MID/__initdask__.py
"""
Created on Wed Feb 19 16:24:03 2020

@author: cmchico
"""
import dask.bag as db
from dask.distributed import Client, progress

client = Client(processes = True,n_workers=12)
client

'\nCreated on Wed Feb 19 16:24:03 2020\n\n@author: cmchico\n'

C:\ProgramData\Anaconda3\lib\site-packages\distributed\bokeh\core.py:57: UserWarning:


Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.



Client Scheduler: tcp://127.0.0.1:50321 Dashboard: http://127.0.0.1:50324/status,Cluster Workers: 12 Cores: 12 Memory: 34.14 GB


In [3]:
import itertools

In [4]:
from fbprophet.plot import add_changepoints_to_plot 

In [5]:
path = 'D:/BUDS_MID/OJT ATM/ATM Forecasting Data New/'

In [6]:
atm_profile = pd.read_excel(path+'Actual/ATM Profile.xlsx', dtype={'TID':'str'})

In [7]:
rmse = pd.concat([pd.read_csv(path+'/Prophet Train/'+file, usecols=['tid','model','dev_rmse','val_rmse']) for file in os.listdir(path+'/Prophet Train/')],ignore_index=True)
rmse['mean_rmse'] = rmse[['dev_rmse','val_rmse']].mean(axis=1) 
rmse.info()

perf =pd.read_pickle(path+'89 Threshold Recommendations.pkl')
best = pd.read_csv(path+'83 Best Models.csv')
data = pd.read_pickle(path+'Actual/82 Aggregated TIDBR WDL.pkl')
HP = pd.read_csv(path + 'Actual/Holidays_ALL.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8262 entries, 0 to 8261
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tid        8262 non-null   object 
 1   model      8262 non-null   object 
 2   dev_rmse   8262 non-null   float64
 3   val_rmse   8262 non-null   float64
 4   mean_rmse  8262 non-null   float64
dtypes: float64(3), object(2)
memory usage: 322.9+ KB


In [113]:
HP1 =  pd.read_csv(path+'Actual/Holidays_ALL.csv',index_col='ds',parse_dates=['ds'])
# HP1  =
HP1['event']  = HP1.holiday.apply(lambda x: 'Payroll' if 'Payroll' in x else 'Holiday')
HP1.reset_index(inplace=True)
HP1.info()

holi = HP1.pivot_table(index='ds',columns='event',values='holiday',aggfunc='count').reset_index()
holi['ds_next'] = holi.ds+pd.to_timedelta('1 days')
holi['event'] = holi[['Holiday','Payroll']].fillna(0).apply(lambda x: "Holiday and Payroll" if sum(x) == 2 else "")

holi[holi[['Holiday','Payroll']].isnull().sum(axis=1)==2]
holi.loc[holi.Holiday.isnull(),'event']='Payroll'
holi.loc[holi.Payroll.isnull(),'event']='Holiday'
holi = HP1[['ds','upper_window','lower_window']].copy()
holi['event'] = HP1.holiday
holi

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451 entries, 0 to 450
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ds            451 non-null    datetime64[ns]
 1   holiday       451 non-null    object        
 2   lower_window  451 non-null    int64         
 3   upper_window  451 non-null    int64         
 4   event         451 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 17.7+ KB


event,ds,Holiday,Payroll,ds_next,event


,ds,upper_window,lower_window,event
0,2017-01-01,0,-1,New Year's Day
1,2018-01-01,0,-1,New Year's Day
2,2019-01-01,0,-1,New Year's Day
3,2020-01-01,0,-1,New Year's Day
4,2021-01-01,0,-1,New Year's Day
...,...,...,...,...
446,2021-08-30,1,-1,2nd Monthly Payroll 30th
447,2021-09-30,1,-1,2nd Monthly Payroll 30th
448,2021-10-29,1,-1,2nd Monthly Payroll 30th
449,2021-11-29,1,-1,2nd Monthly Payroll 30th


In [9]:
# val_length= timedelta(days=datetime.date(2019,12,18) - datetime.date(2018,12,31))


In [66]:
train_dates = pd.date_range(start='2017-01-01', end='2019-12-31')
val_dates = pd.date_range(start='2019-01-01', end='2019-12-31')

dev_length =730 #2yrs
val_length= (datetime.date(2019,12,18) - datetime.date(2018,12,31)).days
period = '1 days'
horizon = 14

data_train = data[data.DATE.isin(train_dates)][['DATE','TID_BR','WDL_AMT_FIN']].copy()
data_train.columns=['ds','tid','y']

week = [0,2,3]
mon = [0,2,3,4,5,6]
hol = ['HP']
holidays=holi[holi.ds.isin(train_dates)]

scenario =  pd.DataFrame(list(itertools.product(hol,week,mon)),columns=['hol','week','mon'])
scenario['model'] = scenario.hol + scenario.week.astype('str') +scenario.mon.astype(str)
scenario['hol_num'] = scenario['hol'].map({'HP':0,'OP':1})

print('\ntotal scenario: {}'.format(scenario.shape[0]))



total scenario: 18


In [67]:
def prep_viz(model):
#     model['tid_name'] = atm_profile[(atm_profile.TID==atm) | (atm_profile['Branch Code Orig']==atm)]['ATM Name'].values[0]
    model['tid_name'] = 'epson'
    model['dev_rmse'] = np.sqrt(model[model.index<'2019-01-01'].se.mean())
    model['val_rmse'] = model[model.index>='2019-01-01'].groupby('pred_day').se.mean().agg(np.sqrt).mean()
    model['%error'] = (model.error/model.y).fillna(0).abs()
    model['dev_mape'] = model[(model['%error'] != np.inf) & (model.index<'2019-01-01')]['%error'].mean()
    model['val_mape'] = model[(model['%error'] != np.inf) & (model.index>='2019-01-01')]['%error'].mean()

    dev = model[model.index<'2019-01-01'].copy()
    val = model[(model.index>='2019-01-01')].copy()
    val[['upper','lower','median']] = val[['yhat','pred_day']].pivot_table(index=val.index,columns='pred_day',values='yhat').agg(['max','min','median'],axis=1)
    all_ = pd.concat([dev,val[val.pred_day=='day 0']])
    all_.upper = all_.upper.fillna(all_.yhat)
    all_.lower = all_.lower.fillna(all_.yhat)
    
    return all_

In [68]:
perf2 = perf[['tid','rmse','y_ave']].drop_duplicates().reset_index(drop=True)
perf2['rmse_yave'] = perf2.rmse/perf2.y_ave
perf2.tid = perf2['tid'].astype('str')
perf2 = perf2.merge(atm_profile[['TID','Branch Code Orig','ATM Name']], how='left', left_on='tid',right_on='TID')
perf2 = perf2.sort_values('rmse_yave',ascending=False)
perf2

,tid,rmse,y_ave,rmse_yave,TID,Branch Code Orig,ATM Name
214,91018953,969917.94,985071.23,0.98,91018953,BR088 OFF,EPSON LIPA OFF 1_B
50,91016261,1008133.87,1138152.71,0.89,91016261,BR378 OFF,TELETECH 5 ECOM 2 OFF_A
189,91018354,757835.78,865693.73,0.88,91018354,BR897 OFF,TOSHIBA INFORMATION EQPT 1 OFF
182,91018288,806584.65,926461.54,0.87,91018288,BR069 OFF,SANKYO SEIKI (NIDEC SUBIC) OFF_B
192,91018361,952804.00,1101035.33,0.87,91018361,BR897 OFF,TOSHIBA INFORMATION EQUIPMENT PHILS-CIP SITE 2...
...,...,...,...,...,...,...,...
416,BR314,1408488.73,9140137.32,0.15,NaN,NaN,NaN
348,BR121,1175525.52,7680179.20,0.15,NaN,NaN,NaN
336,BR077,1389739.95,10238066.10,0.14,NaN,NaN,NaN
449,BR389,1067824.11,8037149.86,0.13,NaN,NaN,NaN


In [69]:
perf2[perf2['Branch Code Orig'] == 'BR088 OFF']

,tid,rmse,y_ave,rmse_yave,TID,Branch Code Orig,ATM Name
214,91018953,969917.94,985071.23,0.98,91018953,BR088 OFF,EPSON LIPA OFF 1_B
163,91017939,917849.78,1135796.87,0.81,91017939,BR088 OFF,FURUKAWA OFF_A
215,91018956,934178.25,1219576.07,0.77,91018956,BR088 OFF,EPSON LIPA OFF 2_B
263,91019673,705324.73,923026.21,0.76,91019673,BR088 OFF,YAMAHA LIPA OFF
302,91019999,1389155.96,2067072.65,0.67,91019999,BR088 OFF,EPSON LIPA OFF 3_B
274,91019761,509248.09,1254592.88,0.41,91019761,BR088 OFF,TELETECH LIPA BATANGAS OFF_A
260,91019656,900019.35,2757071.23,0.33,91019656,BR088 OFF,MARY MEDIATRIX HOSPITAL OFF
195,91018425,380038.16,1585515.38,0.24,91018425,BR088 OFF,NESTLE LIPA OFF


In [70]:
# perf2.sort_values('rmse_yave',ascending=False)
# epson lipa
br = ['91018953','91018956','91019999']

In [72]:
# holi["Holiday"] = 1*(holi.Holiday > 1)
# holi["Payroll"] = 1*(holi.Payroll > 1)
# holi

In [76]:
atm = '91018953'
series = data_train[data_train.tid==atm][['ds','y']].reset_index(drop=True)
week = best[best.tid==atm].week.to_list()[0]
mon = best[best.tid==atm].mon.to_list()[0]
print(week,mon)

3 2


In [114]:
HP = HP1.copy()
HP['include'] = "Yes"
HP.loc[HP.holiday.str.contains("Payroll"),"include"]="No"
HP.loc[HP.holiday.str.contains("15"),"include"]="Yes"
HP.loc[HP.holiday.str.contains("30"),"include"]="Yes"
HP.loc[HP.holiday.str.contains("Payroll"),"holiday"]="Payroll"

In [77]:
# series = series.merge(holi[['ds','Holiday','Payroll']], how="left").fillna(0)
# series['Payroll_'] = series.Payroll

In [78]:
start=time()
# model,val = train(series[0:0+dev_length],series[0+dev_length:],HP[HP.include=="Yes"].copy(),week,mon,cpoint=0.05,
#                  add_regressor_col='Payroll_')
fit, model,rmse = predict(atm,series,dev_length,val_length,HP,week,mon, horizon,cpoint=0.5)
model = model.set_index('ds')
print(timedelta(seconds=time()-start))


ATM:91018953  RMSE:502426.1766458292
RMSE 704.6436192728289
0:03:16.486504


In [84]:
rmse

,ds,pred_day,se
0,1,day 0,374227.50
1,2,day 0,401993.15
2,3,day 0,388479.64
3,4,day 0,458670.18
4,5,day 0,575850.60
...,...,...,...
163,12,day 5,721253.75
164,12,day 6,706822.40
165,12,day 7,682159.97
166,12,day 8,656280.82


In [122]:
start=time()
# model,val = train(series[0:0+dev_length],series[0+dev_length:],HP[HP.include=="Yes"].copy(),week,mon,cpoint=0.05,
#                  add_regressor_col='Payroll_')
fit, model,rmse = predict(atm,series,dev_length,val_length,HP,week,mon, horizon,cpoint=0.5)
model = model.set_index('ds')
print(f'RMSE {np.sqrt(rmse.se.mean())}')
print(timedelta(seconds=time()-start))


ATM:91018953  RMSE:509461.1345410941
RMSE 523448.13179886306
0:03:47.665165


In [40]:
best[best.tid.isin(br)]

,Unnamed: 0,tid,best_model,week,mon
214,214,91018953,HP32,3,2
215,215,91018956,HP32,3,2
302,302,91019999,HP34,3,4


In [115]:
br = ['91018953','91018956','91019999']
series = data_train[data_train.tid.isin(br)][['ds','y']].groupby('ds')[['y']].sum().reset_index()
week = 3
mon = 2

In [106]:
start=time()
# model,val = train(series[0:0+dev_length],series[0+dev_length:],HP[HP.include=="Yes"].copy(),week,mon,cpoint=0.05,
#                  add_regressor_col='Payroll_')
fit, model,rmse = predict('epson',series,dev_length,val_length,HP,week,mon, horizon,cpoint=0.5)
model = model.set_index('ds')
# print(f'RMSE {np.sqrt(rmse.se.mean())}')
print(timedelta(seconds=time()-start))


ATM:epson  RMSE:1494447.8715486918
0:02:46.367092


In [116]:
start=time()
# model,val = train(series[0:0+dev_length],series[0+dev_length:],HP[HP.include=="Yes"].copy(),week,mon,cpoint=0.05,
#                  add_regressor_col='Payroll_')
fit, model,rmse = predict('epson',series,dev_length,val_length,HP,week,mon, horizon,cpoint=0.5)
model = model.set_index('ds')
# print(f'RMSE {np.sqrt(rmse.se.mean())}')
print(timedelta(seconds=time()-start))


ATM:epson  RMSE:1617984.6221977018
0:03:20.852615


In [45]:
start=time()
# model,val = train(series[0:0+dev_length],series[0+dev_length:],HP[HP.include=="Yes"].copy(),week,mon,cpoint=0.05,
#                  add_regressor_col='Payroll_')
fit, model,rmse = predict('epson',series,dev_length,val_length,HP,week,mon, horizon,cpoint=0.5)
model = model.set_index('ds')
# print(f'RMSE {np.sqrt(rmse.se.mean())}')
print(timedelta(seconds=time()-start))


ATM:epson  RMSE:1535109.3700819565


NameError: name 'val' is not defined

In [80]:
model

,y,yhat,pred_date,pred_day,error,se,tid
ds,,,,,,,
2017-01-01,0.00,68984.49,2018-12-31,day 0,-68984.49,4758860258.02,91018953
2017-01-02,8900.00,75037.08,2018-12-31,day 0,-66137.08,4374112775.79,91018953
2017-01-03,61800.00,292452.78,2018-12-31,day 0,-230652.78,53200703666.89,91018953
2017-01-04,197000.00,298116.74,2018-12-31,day 0,-101116.74,10224595642.41,91018953
2017-01-05,719400.00,406739.66,2018-12-31,day 0,312660.34,97756487520.04,91018953
...,...,...,...,...,...,...,...
2019-12-27,0.00,499618.80,2019-12-17,day 9,-499618.80,249618940744.12,91018953
2019-12-28,0.00,346681.47,2019-12-17,day 10,-346681.47,120188039510.31,91018953
2019-12-29,0.00,81841.96,2019-12-17,day 11,-81841.96,6698106289.57,91018953


In [104]:
model

,y,yhat,pred_date,pred_day,error,se,tid,tid_name,dev_rmse,val_rmse,%error,dev_mape,val_mape
ds,,,,,,,,,,,,,
2017-01-01,0.00,68984.49,2018-12-31,day 0,-68984.49,4758860258.02,91018953,epson,419718.44,519743.69,inf,2.93,5.46
2017-01-02,8900.00,75037.08,2018-12-31,day 0,-66137.08,4374112775.79,91018953,epson,419718.44,519743.69,7.43,2.93,5.46
2017-01-03,61800.00,292452.78,2018-12-31,day 0,-230652.78,53200703666.89,91018953,epson,419718.44,519743.69,3.73,2.93,5.46
2017-01-04,197000.00,298116.74,2018-12-31,day 0,-101116.74,10224595642.41,91018953,epson,419718.44,519743.69,0.51,2.93,5.46
2017-01-05,719400.00,406739.66,2018-12-31,day 0,312660.34,97756487520.04,91018953,epson,419718.44,519743.69,0.43,2.93,5.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-27,0.00,499618.80,2019-12-17,day 9,-499618.80,249618940744.12,91018953,epson,419718.44,519743.69,inf,2.93,5.46
2019-12-28,0.00,346681.47,2019-12-17,day 10,-346681.47,120188039510.31,91018953,epson,419718.44,519743.69,inf,2.93,5.46
2019-12-29,0.00,81841.96,2019-12-17,day 11,-81841.96,6698106289.57,91018953,epson,419718.44,519743.69,inf,2.93,5.46


In [117]:
# forecast=model.predict(series[0+dev_length:])
# model.plot_components(forecast);

all_ = prep_viz(model)
fviz.visual(all_,holi, path+'Prophet Visuals/');

In [110]:
holi['ds_next'] = holi['ds'] + pd.to_timedelta('1 days')

In [43]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar  8 15:36:44 2020

@author: cmchico
"""

from fbprophet import Prophet
import numpy as np
import dask.bag as db
import pandas as pd

def train(series,val_dates,holidays,weekly=0,monthly=0,cpoint=0.05, add_regressor_col=''):
    series = series.copy()
    val_dates = val_dates.copy()
    #define model
    modelf = Prophet(interval_width = 0.95,
                    daily_seasonality = False,
                    weekly_seasonality = False,
                    yearly_seasonality = True,
                    holidays = holidays,
                    changepoint_prior_scale = cpoint)
    if weekly != 0: modelf.add_seasonality(name='weeklyx', period = 7, fourier_order = weekly)
    if monthly != 0: modelf.add_seasonality(name='monthlyx', period = 30.5, fourier_order = monthly)
    if add_regressor_col !='': modelf.add_regressor(add_regressor_col)
    
    modelf.fit(series)
    
    val_dates['yhat'] = modelf.predict(val_dates).yhat.values.clip(min=0)
    val_dates['pred_date'] = series.tail(1).ds.dt.date.values.tolist()[0]
    val_dates['pred_day'] = ['day '+str(i) for i in range(val_dates.shape[0])]
    val_dates['error'] = val_dates.y - val_dates.yhat
    val_dates['se'] = val_dates.error**2
    
    return modelf,val_dates

def predict(atm,series,dev_length,val_length, HP, week,mon, horizon,cpoint = 0.05,add_regressor_col=""):

    #Predict for development dataset
    fitted_model,model1 = train(series[0:dev_length],series[0:dev_length], HP, week, mon, cpoint=cpoint)
    model1['pred_day'] = 'day 0'
    model1['tid'] = atm
    rmse1 = model1.groupby([model1.ds.dt.month,'pred_day']).se.mean().agg(np.sqrt).reset_index()
    
    dbseries = db.from_sequence([series[i:i+dev_length] for i in range(val_length)])
    dbforecast = db.from_sequence([series[i+dev_length:i+dev_length+horizon]for i in range(val_length)])

    dbmaster = db.map(train,dbseries,dbforecast,HP,week,mon, cpoint=cpoint,add_regressor_col=add_regressor_col)

    modelf =dbmaster.compute()
    model = pd.concat([modelf[i][1] for i in range(len(modelf))]) 
    model['tid'] = atm
    
    rmse = model.groupby([model.ds.dt.month,'pred_day']).se.mean().agg(np.sqrt).reset_index()
    
    
    print(f'\nATM:{atm}  RMSE:{rmse.se.mean()}')
    last_model = modelf[len(modelf)-1][0]
    return last_model, pd.concat([model1,model]),pd.concat([rmse1,rmse])


'\nCreated on Sun Mar  8 15:36:44 2020\n\n@author: cmchico\n'